In [42]:
# 安装langchain
!pip install langchain
!pip install langgraph
!pip install langchain_openai
!pip install grandalf

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/61/30/44c7eb0a952478dbb5f2f67df806686d6a7e4b19f6204e091c4f49dc7c69/grandalf-0.8-py3-none-any.whl (41 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/9d/ea/6d76df31432a0e6fdf81681a895f009a4bb47b3c39036db3e1b528191d52/pyparsing-3.1.2-py3-none-any.whl (103 kB)


In [22]:
from langchain_openai import ChatOpenAI
from typing import Annotated, List, Sequence, TypedDict
import operator

# 初始化模型
llm = ChatOpenAI(
    openai_api_key='sk-cHI6zHo4Mye29vcO262b9dEe821a4dBb93253bAd75Ee9e09',
    openai_api_base='http://192.168.1.8:15000/v1',
    model_name='gpt-4-0125-preview',
    temperature=0.0,
    max_tokens=None
)

from typing import TypedDict, Optional

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    history: Annotated[Sequence[BaseMessage], operator.add]
    result: str


In [24]:
from langgraph.graph import StateGraph, END
import time
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 创建图
graph = StateGraph(State)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个助手",
        ),
        MessagesPlaceholder(variable_name="history"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chat_agent = prompt | llm

# 对话节点
def chat(state):
    # 模拟数据处理逻辑
    result = chat_agent.invoke(state)
    print(result)
    state["history"] += [HumanMessage(content=result.content)]
    state['result'] = result.content
    return state

graph.add_node("chat", chat)


# 条件节点来决定是否继续聊天
def should_continue(state: State):
    return END

graph.add_conditional_edges(
    "chat", 
    should_continue, 
    {"chat": "chat", END: END}
)


# 设置入口点
graph.set_entry_point("chat")

# 编译图形
app = graph.compile()

# 初始化状态
initial_state = State(history=[], messages=[HumanMessage(content="你好")], result="")

# 运行图处理流式数据
state = initial_state
for s in app.stream(state):
    print(s)

content='你好！有什么可以帮助你的吗？' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 19, 'total_tokens': 36}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f33d03b8-7bd1-4749-bb61-b387c71bf80f-0'
{'chat': {'messages': [HumanMessage(content='你好')], 'history': [HumanMessage(content='你好！有什么可以帮助你的吗？')], 'result': '你好！有什么可以帮助你的吗？'}}


In [ ]:
# 还有一种方法将图保存到数据库中，历史记录持久化到数据库中
# 请注意需要用下面代码替换或者插入到上面代码对应的位置
from psycopg_pool import AsyncConnectionPool, ConnectionPool

pool = ConnectionPool(
    # Example configuration
    conninfo="postgresql://username:password@ip:5433/app",
    max_size=20,
)

memory = PostgresSaver(
    serializer=PickleCheckpointSerializer(),
    sync_connection=pool,
)

graph = workflow.compile(checkpointer=memory)
# 预测的时候加上会话id
thread = {"configurable": {"thread_id": 1}}
for s in graph.stream({
    "messages": [
        HumanMessage(content=meeting.question)
    ], "history": history2
}, thread):
    print(s)